In [4]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

In [5]:
from cdqa.utils.download import download_model, download_bnpp_data

download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')


100% [..........................................................................] 7025349 / 7025349
100% [......................................................................] 438036612 / 438036612

In [6]:
df = pd.read_csv('./data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

,date,title,category,link,abstract,paragraphs
0,13.05.2019,The banking jobs : Assistant Vice President – ...,Careers,https://group.bnpparibas/en/news/banking-jobs-...,Within the Group’s Corporate and Institutional...,[I manage a team in charge of designing and im...
1,13.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[With François Hollande, Chairman of French fo..."
2,13.05.2019,"""The bank with an IT budget of more than EUR6 ...",Group,https://group.bnpparibas/en/news/the-bank-budg...,"Interview with Jean-Laurent Bonnafé, Director ...","[We did the groundwork between 2012 and 2016, ..."
3,10.05.2019,BNP Paribas at #VivaTech : discover the progra...,Innovation,https://group.bnpparibas/en/news/bnp-paribas-v...,"From Thursday 16 to Saturday 18 May 2019, join...","[As part of the ‘United Tech of Europe’ theme,..."
4,10.05.2019,When Artificial Intelligence participates in r...,Careers,https://group.bnpparibas/en/news/artificial-in...,As the competition to attract talent intensifi...,[Online recruitment is already the norm. Accor...


In [8]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa_vCPU-sklearn.joblib')

# Send model to GPU
cdqa_pipeline.cuda()

cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True,
                         fp16=False, gradient_accumulation_steps=1,
                         learning_rate=3e-05, local_rank=-1, loss_scale=0,
                         max_answer_length=30, n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   token_pattern='(?u)\\b\\w\\w+\\b',
             

In [9]:
query = 'Since when does the Excellence Program of BNP Paribas exist?'
prediction = cdqa_pipeline.predict(query)

In [10]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: Since when does the Excellence Program of BNP Paribas exist?
answer: January 2016
title: BNP Paribas’ commitment to universities and schools
paragraph: Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.


In [20]:
df.paragraphs[10]

['BNP Paribas is a leading bank in Europe with an international reach. It has a presence in 72 countries, with more than 202,000 employees, of which more than 154,000 in Europe. The Group has key positions in its three main activities: Domestic Markets and International Financial Services (whose retail-banking networks and financial services are covered by Retail Banking & Services) and Corporate & Institutional Banking, which serves two client franchises: corporate clients and institutional investors. The Group helps all its clients (individuals, community associations, entrepreneurs, SMEs, corporates and institutional clients) to realise their projects through solutions spanning financing, investment, savings and protection insurance.',
 'BNP Paribas is rolling out its integrated retail-banking model in Mediterranean countries, in Turkey, in Eastern Europe and a large network in the western part of the United States. In its Corporate & Institutional Banking and International Financia

## PDF to Paragraph

In [21]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

In [ ]:
# Download model
download_model(model='bert-squad_1.1', dir='./models')

In [22]:
# Download pdf files from BNP Paribas public news
def download_pdf():
    import os
    import wget
    directory = './data/pdf/'
    models_url = [
      'https://invest.bnpparibas.com/documents/1q19-pr-12648',
      'https://invest.bnpparibas.com/documents/4q18-pr-18000',
      'https://invest.bnpparibas.com/documents/4q17-pr'
    ]

    print('\nDownloading PDF files...')

    if not os.path.exists(directory):
        os.makedirs(directory)
    for url in models_url:
        wget.download(url=url, out=directory)

download_pdf()


100% [............................................................................] 776686 / 776686

In [23]:
df = pdf_converter(directory_path='./data/pdf/')
df.head()

2019-10-06 13:45:46,769 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to C:\Users\ASUS\AppData\Local\Temp\tika-server.jar.
2019-10-06 13:46:29,668 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to C:\Users\ASUS\AppData\Local\Temp\tika-server.jar.md5.
2019-10-06 13:46:30,477 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2019-10-06 13:46:35,482 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


,title,paragraphs
0,1q19-pr-12648.pdf,"[FIRST QUARTER , 2019 RESULTS , PRESS RELEAS..."
1,4q17-pr.pdf,"[2017 FULL YEAR RESULTS , PRESS RELEASE , Pa..."
2,4q18-pr2.pdf,"[2018 FULL YEAR RESULTS , PRESS RELEASE , Pa..."


In [25]:
#cdqa_pipeline = QAPipeline(reader='./models/bert_qa_vCPU-sklearn.joblib', max_df=1.0)

# Send model to GPU
#cdqa_pipeline.cuda()

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True,
                         fp16=False, gradient_accumulation_steps=1,
                         learning_rate=3e-05, local_rank=-1, loss_scale=0,
                         max_answer_length=30, n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=2,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_port='', train_batch_size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   token_pattern='(?u)\\b\\w\\w+\\b',
             

In [28]:
query = 'How many contracts did BNP Paribas Cardif sell in 2019?'
prediction = cdqa_pipeline.predict(query)

In [30]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many contracts did BNP Paribas Cardif sell in 2019?
answer: 100,000
title: 4q18-pr2.pdf
paragraph: BNP Paribas Cardif and Matmut (Cardif IARD) is a success with already 100,000 contracts sold as at 31 December 2018. The goal is to multiply by three by 2020 sales of property and casualty contracts and to grow the customer penetration rate from 8% to 12%.  


In [8]:
def test_args_kwargs(arg1, arg2, arg3):
    print("arg1:", arg1)
    print("arg2:", arg2)
    print("arg3:", arg3)
    
    
# first with *args
#args1 = ("two", 3, 5)
# test_args_kwargs(*args1)

# now with **kwargs:
kwargs = (arg3=3, arg2="two", arg1=5)
test_args_kwargs(**kwargs)

SyntaxError: invalid syntax (<ipython-input-8-3bc45bb8ecfa>, line 12)

In [4]:
def test_kwargs(**kwargs):
    kwargs_processor = {key: value for key, value in kwargs.items()}
    return print(kwargs_processor)

test_kwargs(name="yasoob",type1="hunkey",abc="123")

{'name': 'yasoob', 'type1': 'hunkey', 'abc': '123'}
